# Roberta Large Fine Tuned on RACE

## Model description

This model is a fine-tuned model of Roberta-large applied on RACE

#### How to use


In [ ]:
import paddle
from paddlenlp.transformers import AutoModel

model = AutoModel.from_pretrained("LIAMF-USP/roberta-large-finetuned-race")
input_ids = paddle.randint(100, 200, shape=[1, 20])
print(model(input_ids))


## Training data

The initial model was [roberta large model](https://huggingface.co/roberta-large) which was then fine-tuned on [RACE dataset](https://www.cs.cmu.edu/~glai1/data/race/)

## Training procedure

It was necessary to preprocess the data with a method that is exemplified for a single instance in the _How to use_ section. The used hyperparameters were the following:

| Hyperparameter | Value |
|:----:|:----:|
| adam_beta1                  | 0.9      |
| adam_beta2                  | 0.98     |
| adam_epsilon                | 1.000e-8 |
| eval_batch_size             | 32       |
| train_batch_size            | 1        |
| fp16                        | True     |
| gradient_accumulation_steps | 16       |
| learning_rate               | 0.00001  |
| warmup_steps                | 1000     |
| max_length                  | 512      |
| epochs                      | 4        |

## Eval results:
| Dataset Acc | Eval | All Test |High School Test |Middle School Test |
|:----:|:----:|:----:|:----:|:----:|
|      | 85.2 | 84.9|83.5|88.0|

**The model was trained with a Tesla V100-PCIE-16GB**
> 此模型来源于：[https://huggingface.co/LIAMF-USP/roberta-large-finetuned-race](https://huggingface.co/https://huggingface.co/LIAMF-USP/roberta-large-finetuned-race)

> 此模型来源于：[LIAMF-USP/roberta-large-finetuned-race](https://huggingface.co/LIAMF-USP/roberta-large-finetuned-race)